## chapter10 Sequence hacking,hashing and slicing

In [ ]:
from array import array
import reprlib
import math
import functools
import operator

class Vector(object):
    typecode = 'd'
    def __init__(self,components):
        self._components = array(self.typecode,components)
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        components = reprlib.repr(self._components)
        print(components)
        components = components[components.find('[]'):-1]
        return 'Vector({})'.format(components)
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    def __eq__(self,other):
        return tuple(self)==tuple(other)
    def __abs__(self):
        return math.sqrt(sum(x*x for x in self))
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls,octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    def __len__(self):
        return len(self._components)
    def __getitem__(self,index):
        return self._components[index]
    
    shortcut_names = 'xyzt'
    def __getattr__(self,name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
            msg = '{.__name__!r} object has no attribute {!r}'
            raise AttrbuteError(msg.format(cls,name))
    '''
    def __setattr__(self,name,value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__,attr_name=name)
                raise AttributeError(msg)
            super().__setattr__(name,value)
    '''
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor,hashes,0)

In [ ]:
v1 = Vector([3,4,5])
v1[0:3]

In [ ]:
str(v1)

In [ ]:
repr(v1)

In [ ]:
abs(v1)

In [ ]:
v1_bytes = bytes(v1)
print(v1_bytes)
v2 = Vector.frombytes(v1_bytes)
print(v2)

In [ ]:
print(v1.x)

## chapter 12: Inheritance for good or for worse

In [ ]:
class A:
    def ping(self):
        print('ping:', self)
class B(A):
    def pong(self):
        print('pong:', self)
class C(A):
    def pong(self):
        print('PONG:', self)
    
class D(C,B):
    def ping(self):
        super().ping()
        print('post-ping:', self)
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [ ]:
d = D()
d.ping()

In [ ]:
D.__mro__

In [ ]:
d.pingpong()

In [ ]:
d.pong()
C.pong(d)

### mro

In [ ]:
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

In [ ]:
print_mro(bool)